In [2]:
!pip install pyspark

In [3]:
import pyspark
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.storagelevel import StorageLevel

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
produtos = spark.read.csv('/Users/Yuri/Desktop/EXCEL/produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('/Users/Yuri/Desktop/EXCEL/vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('/Users/Yuri/Desktop/EXCEL/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('/Users/Yuri/Desktop/EXCEL/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('/Users/Yuri/Desktop/EXCEL/pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('/Users/Yuri/Desktop/EXCEL/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('/Users/Yuri/Desktop/EXCEL/pedidos.csv', header=True, inferSchema=True)

In [6]:
clientes.show()
pedidos.show()
itens_pedido.show()
pagamentos_pedido.show()

+--------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-----------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|       1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|       8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|      13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|      89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|       4534|           sao paulo|            SP|
|5e274e7a0c3809e14...|57b2a98a409812fe9...|      35182|             timoteo|    

In [7]:
clientes.createOrReplaceTempView('clientes')

spark.sql('CACHE TABLE clientes')
spark.sql('SELECT COUNT(*) FROM clientes').show()
spark.sql('UNCACHE TABLE clientes')

+--------+
|count(1)|
+--------+
|   99441|
+--------+



DataFrame[]

In [45]:
clientes.cache() # persist(storagelevel.memory_and_disk)

clientes.persist(StorageLevel.MEMORY_AND_DISK)

clientes_estado_sp_df = clientes.filter(clientes['estado_cliente'] == 'SP')
clientes_estado_sp_df.show()

clientes.unpersist()

+--------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-----------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|       1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|       8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|      13056|            campinas|            SP|
|fd826e7cf63160e53...|addec96d2e059c80c...|       4534|           sao paulo|            SP|
|b2d1536598b73a9ab...|918dc87cd72cd9f6e...|      18682|    lencois paulista|            SP|
|eabebad39a88bb6f5...|295c05e81917928d7...|       5704|           sao paulo|    

DataFrame[id_cliente: string, id_unico_cliente: string, cep_cliente: int, cidade_cliente: string, estado_cliente: string]

In [9]:
print(itens_pedido.count())
print(pedidos.count())

small_pedidos_df = broadcast(pedidos)

broadcast_data = itens_pedido.join(small_pedidos_df, on='id_pedido')
broadcast_data.show()

112650
99441
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio| preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 06:45:35|  58.9|      13.29|3ce436f183e68e078...|    delivered|2017-09-13 05:59:02|  2017-09-13 06:45:

In [9]:
salario_medio_valor = pagamentos_pedido.agg(avg('valor_pagamento').alias('pagamento_medio')).collect()[0][0]
pedidos_filtrados_df = pagamentos_pedido.filter(pagamentos_pedido['valor_pagamento'] > salario_medio_valor)

salario_medio_df = pagamentos_pedido.select(avg('valor_pagamento').alias('pagamento_medio'))
pedidos_filtrados_join_df = pagamentos_pedido.join(
    salario_medio_df,
    pagamentos_pedido['valor_pagamento'] > salario_medio_df['pagamento_medio']
).select(pagamentos_pedido['*'])

salario_medio_df.show()
pedidos_filtrados_join_df.show()

+------------------+
|   pagamento_medio|
+------------------+
|154.10038041698772|
+------------------+

+--------------------+-------------------+--------------+------------------+---------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|
+--------------------+-------------------+--------------+------------------+---------------+
|1f78449c87a54faf9...|                  1|   credit_card|                 6|         341.09|
|d88e0d5fa41661ce0...|                  1|   credit_card|                 8|         188.73|
|12e5cfe0e4716b59a...|                  1|   credit_card|                10|         157.45|
|8ac09207f415d55ac...|                  1|   credit_card|                 4|         244.15|
|4214cda550ece8ee6...|                  1|   credit_card|                 2|         170.57|
|4d680edbaa7d3d9be...|                  1|   credit_card|                10|         353.09|
|8cd68144cdb62dc0d...|                  1|        boleto|

In [37]:
salario_medio_valor = pagamentos_pedido.agg(avg('valor_pagamento').alias('pagamento_medio')).collect()[0][0]
pedidos_filtrados_df = pagamentos_pedido.filter(pagamentos_pedido['valor_pagamento'] > salario_medio_valor)

print('Plano de execucao - collect')
pedidos_filtrados_df.explain()

salario_medio_df = pagamentos_pedido.select(avg('valor_pagamento').alias('pagamento_medio'))
pedidos_filtrados_join_df = pagamentos_pedido.join(
    salario_medio_df,
    pagamentos_pedido['valor_pagamento'] > salario_medio_df['pagamento_medio']
).select(pagamentos_pedido['*'])

print('Plano de execucao - join')
pedidos_filtrados_join_df.explain()

Plano de execucao - collect
== Physical Plan ==
*(1) Filter (isnotnull(valor_pagamento#114) AND (valor_pagamento#114 > 154.10038041698772))
+- FileScan csv [id_pedido#110,sequencia_pagamento#111,tipo_pagamento#112,parcelas_pagamento#113,valor_pagamento#114] Batched: false, DataFilters: [isnotnull(valor_pagamento#114), (valor_pagamento#114 > 154.10038041698772)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/Yuri/Desktop/EXCEL/pagamentos_pedido.csv], PartitionFilters: [], PushedFilters: [IsNotNull(valor_pagamento), GreaterThan(valor_pagamento,154.10038041698772)], ReadSchema: struct<id_pedido:string,sequencia_pagamento:int,tipo_pagamento:string,parcelas_pagamento:int,valo...


Plano de execucao - join
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [id_pedido#110, sequencia_pagamento#111, tipo_pagamento#112, parcelas_pagamento#113, valor_pagamento#114]
   +- BroadcastNestedLoopJoin BuildRight, Inner, (valor_pagamento#114 > pagamento_medio#1059)
      :

In [38]:
start_time = time.time()

salario_medio_valor = pagamentos_pedido.agg(avg('valor_pagamento').alias('pagamento_medio'))
pedidos_filtrados_join_df = pagamentos_pedido.join(
    salario_medio_df,
    pagamentos_pedido['valor_pagamento'] > salario_medio_df['pagamento_medio']
).select(pagamentos_pedido['*'])

end_time = time.time()
print(end_time - start_time)

0.02866983413696289


In [43]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

start_time = time.time()
def classificar_preco(preco):
    if preco < 50:
        return 'Baixo'
    elif preco >= 50 and preco < 500:
        return 'Medio'
    else: return 'Alto'
classificar_preco_udf = udf(classificar_preco, StringType())

pedidos_classificacao_udf_df = itens_pedido.withColumn('classificacao_preco', classificar_preco_udf(itens_pedido['preco']))
end_time = time.time()
print(end_time - start_time)
pedidos_classificacao_udf_df.explain()

start_time = time.time()
pedidos_classificacao_df = itens_pedido.withColumn('classificacao_preco',
                                                   when(itens_pedido['preco'] < 50, 'Baixo')
                                                   .when((itens_pedido['preco'] >= 50) & (itens_pedido['preco'] < 500), 'Medio')
                                                   .otherwise('Alto'))
end_time = time.time()
print(end_time - start_time)
pedidos_classificacao_df.explain()


0.10129427909851074
== Physical Plan ==
*(1) Project [id_pedido#86, item_id_pedido#87, id_produto#88, id_vendedor#89, data_limite_envio#90, preco#91, valor_frete#92, pythonUDF0#1086 AS classificacao_preco#1085]
+- BatchEvalPython [classificar_preco(preco#91)#1084], [pythonUDF0#1086]
   +- FileScan csv [id_pedido#86,item_id_pedido#87,id_produto#88,id_vendedor#89,data_limite_envio#90,preco#91,valor_frete#92] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/Yuri/Desktop/EXCEL/itens_pedido.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id_pedido:string,item_id_pedido:int,id_produto:string,id_vendedor:string,data_limite_envio...


0.03973507881164551
== Physical Plan ==
*(1) Project [id_pedido#86, item_id_pedido#87, id_produto#88, id_vendedor#89, data_limite_envio#90, preco#91, valor_frete#92, CASE WHEN (preco#91 < 50.0) THEN Baixo WHEN ((preco#91 >= 50.0) AND (preco#91 < 500.0)) THEN Medio ELSE Alto END AS classificacao_preco